In [1]:
import math
import mathutils as mu
import numpy as np

In [2]:
class ShapeParabola: 
    # points and normals for parabola
    
    def __init__(self, focus = 1.):
        self.focus = focus
        
    def point(self, u, v):
        return mu.Vector((
            u,
            v,
            (u*u + v*v)/(4.*self.focus)
        ))
    
    def normal(self, u, v):
        return mu.Vector((
            -u/self.focus,
            -v/self.focus,
            2.        
        )).normalized()

In [3]:
shape = ShapeParabola(1.)

In [4]:
def toText(v):
    return ' '.join([str(c) for c in v])

## normal map (Inventor format)

In [5]:
nMax = 3
mMax = 3
points = [
    (u, v)
    for u in np.linspace(-0.5, 0.5, nMax)
    for v in np.linspace(-0.5, 0.5, mMax)]

In [6]:
normals = [shape.normal(p[0], p[1]) for p in points]

In [7]:
', '.join([toText(n) for n in normals])

'0.2357022762298584 0.2357022762298584 0.9428091049194336, 0.24253563582897186 -0.0 0.9701425433158875, 0.2357022762298584 -0.2357022762298584 0.9428091049194336, -0.0 0.24253563582897186 0.9701425433158875, -0.0 -0.0 1.0, -0.0 -0.24253563582897186 0.9701425433158875, -0.2357022762298584 0.2357022762298584 0.9428091049194336, -0.24253563582897186 -0.0 0.9701425433158875, -0.2357022762298584 -0.2357022762298584 0.9428091049194336'

## mesh (smooth, per vertex normal, obj format)

In [8]:
nMax = 5
mMax = 5
points = [
    (u, v)
    for u in np.linspace(-0.5, 0.5, nMax)
    for v in np.linspace(-0.5, 0.5, mMax)]

In [9]:
vertices = [shape.point(p[0], p[1]) for p in points]
normals = [shape.normal(p[0], p[1]) for p in points]

In [10]:
def toIndex(n, m):
    return n*mMax + m + 1

faces = np.array([
    [(toIndex(n, m), toIndex(n + 1, m), toIndex(n, m + 1)),
     (toIndex(n + 1, m + 1), toIndex(n, m + 1), toIndex(n + 1, m))]      
 for n in range(nMax - 1)
 for m in range(mMax - 1)
])
faces = faces.reshape(-1, 3)

In [11]:
out = ''
out += '\n'.join(["v " + toText(v) for v in vertices]) + '\n'
out += '\n'.join(["vn " + toText(n) for n in normals]) + '\n'
out += "s 1\n"
out += '\n'.join(["f {0}//{0} {1}//{1} {2}//{2}".format(a, b, c) for (a, b, c) in faces])
#print(out)
fileOut = open("parabolaSmooth.obj", "w")
fileOut.write(out)
fileOut.close()

## mesh (flat, per face normal, obj format)

In [12]:
faceNormals = [
    (normals[f[0] - 1] + normals[f[1] - 1] + normals[f[2] - 1]).normalized()
    for f in faces]

In [13]:
out = ''
out += '\n'.join(["v " + toText(v) for v in vertices]) + '\n'
out += '\n'.join(["vn " + toText(n) for n in faceNormals]) + '\n'
out += "s off\n"
out += '\n'.join(["f {0}//{3} {1}//{3} {2}//{3}".format(f[0], f[1], f[2], n) for (n, f) in enumerate(faces, 1)])
#print(out)
fileOut = open("parabolaFlat.obj", "w")
fileOut.write(out)
fileOut.close()